In [7]:
import numpy as np
import pandas as pd
import mediapipe as mp
import cv2
import os

In [8]:
landmarks_list = {
    "left_shoulder": [], "right_shoulder": [],
    "left_elbow": [], "right_elbow": [],
    "left_wrist": [], "right_wrist": [],
    "left_hip": [], "right_hip": [],
    "left_knee": [], "right_knee": [],
    "left_ankle": [], "right_ankle": [],
    "left_heel": [], "right_heel": [],
    "left_foot_index": [], "right_foot_index": [],
}

In [9]:
def angle(p1, p2, p3):
    a = np.array([p1[0], p1[1]])
    b = np.array([p2[0], p2[1]])
    c = np.array([p3[0], p3[1]])

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
        np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180:
        angle = 360 - angle

    return angle

In [10]:
mp_pose = mp.solutions.pose

In [11]:
landmarks = {
    "nose": 0,
    "left_shoulder": 11, "right_shoulder": 12,
    "left_elbow": 13, "right_elbow": 14,
    "left_wrist": 15, "right_wrist": 16,
    "left_hip": 23, "right_hip": 24,
    "left_knee": 25, "right_knee": 26,
    "left_ankle": 27, "right_ankle": 28,
    "left_heel": 29, "right_heel": 30,
    "left_foot_index": 31, "right_foot_index": 32,
}

col_names = []
for i in range(len(landmarks.keys())):
    name = list(landmarks.keys())[i]
    col_names.append(name + "_x")
    col_names.append(name + "_y")
    col_names.append(name + "_z")
    col_names.append(name + "_v")

pose_name = col_names.copy()

pose_name.append("pose")

In [12]:
 mp.solutions.pose

# List to store pose data
pose_list = []

# Main directory for images
main_dir = "./poses_dataset/Images"

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    """
    Calculates the angle ABC (in degrees) given three points A, B, and C.
    A, B, and C should be tuples (x, y, z).
    """
    ba = np.array([a[0] - b[0], a[1] - b[1]])  # Vector BA
    bc = np.array([c[0] - b[0], c[1] - b[1]])  # Vector BC

    if np.linalg.norm(ba) == 0 or np.linalg.norm(bc) == 0:
        return 0  # Return 0 if vector length is zero to avoid division by zero

    # Calculate cosine of angle
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Clip to avoid floating-point errors
    return np.degrees(angle)

# Get list of pose directories (filter only directories)
pose_dir_list = [d for d in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, d))]

# Process each directory and image
for i, pose_dir in enumerate(pose_dir_list):
    images_dir_list = os.listdir(f"{main_dir}/{pose_dir}")  # List files in the directory

    # Filter valid image files
    images_dir_list = [f for f in images_dir_list if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for l, image_file in enumerate(images_dir_list):
        pre_list = []
        image_path = f"{main_dir}/{pose_dir}/{image_file}"

        # Load image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue

        # Process image using MediaPipe Pose
        with mp_pose.Pose(static_image_mode=True, enable_segmentation=True) as pose:
            result = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if not result.pose_landmarks:
                print(f"No pose landmarks detected: {image_path}")
                continue
            
            # Extract landmarks
            for landmark in result.pose_landmarks.landmark:
                pre_list.append(landmark)

        # Check if landmarks are complete
        if len(pre_list) < 33:
            print(f"Incomplete landmarks for: {image_path}")
            continue

        # Calculate angles for specific joints
        try:
            # Define key points for angle calculations
            left_shoulder = (pre_list[11].x, pre_list[11].y)
            left_elbow = (pre_list[13].x, pre_list[13].y)
            left_wrist = (pre_list[15].x, pre_list[15].y)

            right_shoulder = (pre_list[12].x, pre_list[12].y)
            right_elbow = (pre_list[14].x, pre_list[14].y)
            right_wrist = (pre_list[16].x, pre_list[16].y)

            left_hip = (pre_list[23].x, pre_list[23].y)
            left_knee = (pre_list[25].x, pre_list[25].y)
            left_ankle = (pre_list[27].x, pre_list[27].y)

            # Calculate angles
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

            # Collect data for CSV
            tpl = [
                left_elbow_angle,
                right_elbow_angle,
                left_knee_angle,
            ]
            tpl.append(i)  # Add pose label
            pose_list.append(tpl)

        except IndexError:
            print(f"Incomplete pose landmarks for {image_path}")
            continue

# Ensure the output directory exists
os.makedirs("./csv_files/", exist_ok=True)

# Save pose data to CSV
pose_name = ["left_elbow_angle", "right_elbow_angle", "left_knee_angle", "pose_label"]
data_pose = pd.DataFrame(pose_list, columns=pose_name)
data_pose.to_csv("./csv_files/pose_angles.csv", index=False)

print("Pose data with angles successfully saved to './csv_files/pose_angles.csv'")


No pose landmarks detected: ./poses_dataset/Images/Goddess/46 - Copy.jpg
No pose landmarks detected: ./poses_dataset/Images/Goddess/46.jpg
Pose data with angles successfully saved to './csv_files/pose_angles.csv'
